In [1]:
import openai
openai.api_key = open("key.txt","r").read().strip('\n')

In [2]:
completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages = [{"role": "user", "content": "How can i use Chatgpt-api"}]
)

print(completion)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "You can use the Chatbot API provided by OpenAI to integrate GPT-3 into your applications, products, or services. Here are the general steps:\n\n1. Signing Up: First, you need to sign up on OpenAI's website and get access to the API. \n\n2. Read the Documentation: OpenAI provides detailed documentation on how to use the API. The documentation gives you general instruction, methods, objects, and error codes you need to understand to be able to use the API effectively.\n\n3. Install Required Libraries: You will need the 'openai' library in Python to use the API. \n   You can install it with pip:\n     pip install openai \n\n4. Use the API Key: After signing up, you will be provided an API key. All your requests to the API have to include this key. Sample Python code:\n   \n    ```\n    import openai\n\n    openai.api_key = 'your-api-key'\n\n    response = openai.ChatCompletion.cr

In [7]:
reply = completion['choices'][0].message.content
print(reply)

You can use the Chatbot API provided by OpenAI to integrate GPT-3 into your applications, products, or services. Here are the general steps:

1. Signing Up: First, you need to sign up on OpenAI's website and get access to the API. 

2. Read the Documentation: OpenAI provides detailed documentation on how to use the API. The documentation gives you general instruction, methods, objects, and error codes you need to understand to be able to use the API effectively.

3. Install Required Libraries: You will need the 'openai' library in Python to use the API. 
   You can install it with pip:
     pip install openai 

4. Use the API Key: After signing up, you will be provided an API key. All your requests to the API have to include this key. Sample Python code:
   
    ```
    import openai

    openai.api_key = 'your-api-key'

    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant."},
 

In [8]:
!pip install python-dotenv
!pip install matplotlib
!pip install pydantic
!pip install pyecharts
!pip install fastapi

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [12]:
#PLOT GPT generated top station data
import re
import matplotlib.pyplot as plt
from pyecharts.charts import Bar, Page, Pie, Grid
from pyecharts import options as opts



# Define the function to plot the gpt data
def plot_gpt_data_to_html(filtered_gpt_data):
    data = filtered_gpt_data

    # Updated pattern to handle both data formats
    pattern = r'(\w[\w\s,]*[^\s])\s\((\d+)\)|\(([\w\s,]+), (\d+)\)'
    matches = re.findall(pattern, data)

    # Extracting station names and frequencies considering both matching groups from the updated pattern
    stations = [match[0] if match[2] == '' else match[2] for match in matches]
    frequencies = [int(match[1]) if match[3] == '' else int(match[3]) for match in matches]

    bar_chart = (
        Bar()
        .add_xaxis(stations)
        .add_yaxis("Frequenz", frequencies, tooltip_opts=opts.TooltipOpts(is_show=True, trigger="axis", axis_pointer_type="cross"))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="Top 10 Betroffenen Stationen"),  
            xaxis_opts=opts.AxisOpts(
                name="Stationen",  # X-Axis title
                axislabel_opts=opts.LabelOpts(
                    rotate=45,   # Try 45-degree rotation
                    font_size=8  # Further reduce font size
                )
            ),
            yaxis_opts=opts.AxisOpts(
                name="Frequenz",  # Y-Axis title
            )
        )
    )

    return bar_chart.render_notebook()



# Define the function to plot the line data
def plot_line_data_to_html(lines_data_list):
    data = lines_data_list
    page = Page(layout=Page.SimplePageLayout)
    bars = []

    for idx, week_data in enumerate(data, 1):
        labels, values = zip(*week_data)
        bar_chart = (
            Bar()
            .add_xaxis(list(labels))
            .add_yaxis(f"Woche {idx}", list(values), label_opts=opts.LabelOpts(is_show=False))
            .set_global_opts(
                title_opts=opts.TitleOpts(title=f"Woche {idx} Vorfälle je Linie"),
                xaxis_opts=opts.AxisOpts(
                    name="Linie",  # X-Axis title
                    axislabel_opts=opts.LabelOpts(rotate=45)  # Optional: Rotate x-axis labels if needed
                ),
                yaxis_opts=opts.AxisOpts(
                    name="Anzahl Vorfälle"  # Y-Axis title
                )
            )
        )
        bars.append(bar_chart)

    page.add(*bars)
    
    print("Generating plot...")
    # Return the embeddable HTML
    #return page.render_embed()
    return page.render_notebook()

# Define the function to plot the accident data
def plot_accident_data_to_html(accident_data_list):
    data = accident_data_list

    def generate_pie(data_item, title):
        categories = list(data_item.keys())
        values = list(data_item.values())
        pie = (
            Pie()
            .add("", [list(z) for z in zip(categories, values)])
            .set_global_opts(title_opts=opts.TitleOpts(title=title), legend_opts=opts.LegendOpts(orient="vertical", pos_top="15%", pos_left="85%"))
            .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))  # Displaying category name and percentage
        )
        return pie

    page = Page()
    for index, data_item in enumerate(data):
        page.add(generate_pie(data_item, f"Woche {index+1}"))
    
    

        
    # Return the embeddable HTML
    #return page.render_embed()
    return page.render_notebook()


In [49]:
import json
from dotenv import load_dotenv
from datetime import datetime, timedelta
import openai
import os
from pyecharts import *
#from plot import plot_line_data_to_html, plot_accident_data_to_html, plot_gpt_data_to_html 

load_dotenv()
# Configure the OpenAI API client
#openai.api_type = os.getenv("OPENAI_API_TYPE")
#openai.api_version = os.getenv("OPENAI_API_VERSION")
#openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
#openai.api_key = os.getenv("OPENAI_API_KEY")

# Define the function to generate the response
#def generate_response(resulted_data):
#    response = openai.ChatCompletion.create(
#        model="gpt-4",
        #engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
#        temperature = 0.4,
#        messages=[
#            {"role": "system", "content": "Du bist ein Analyse-Chatbot. Aus den bereitgestellten JSON-Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-JSON-Daten:"+ resulted_data},
#            {"role": "user", "content": "Im JSON-Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
#            }
#        ]
#    )
#    return response['choices'][0]['message']['content']

def generate_response(resulted_data):
    response = openai.ChatCompletion.create(
    model="gpt-4-0314",
    # engine=os.getenv("DEPLOYMENT_NAME"), # The deployment name you chose when you deployed the GPT-35-Turbo or GPT-4 model.
    temperature=0,
    messages=[
      {"role": "user",
       "content": "Du bist ein Analyst. Aus den bereitgestellten Daten antwortest du auf Nutzerfragen, um Statistiken basierend auf Benutzereingaben zu erstellen. Dies sind die Kontext-List-Daten:" + resulted_data + "Im Datenkontext der Wiener-Linie sind unter Titel betroffene Linien und unter 'Beschreibung' betroffene Stationen verzeichnet. Welche 10 Stationen sind am häufigsten betroffen? Geben Sie nur in diesem Format aus: (Stationsname, Gesamtzahl der Vorfälle). Zum Beispiel: (Rotkreuzplatz, 10)."
      }
    ]
  )
    return response['choices'][0]['message']['content']


# Define the function to filter the accident data
def get_incident_data(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    #gpt filter
    filtered_incidents = [
        {
            'title': entry['title'],
            'description': entry['description']
        } 
        for entry in data['incidents'] 
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime
    ]
        
    resulted_data = json.dumps(filtered_incidents)
    
    print(resulted_data)
    
    #filtered_gpt_data = ''
    filtered_gpt_data = generate_response(resulted_data)
    
    return filtered_gpt_data



# Define the function to filter the line data 
def get_top10_lines(data, start_time, end_time):        
    # Convert the provided timestamp range strings to datetime objects
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)
    
    filtered_lines_incidents = []
    for entry in data['incidents']:
        if start_datetime <= datetime.strptime(entry['start'], date_format) <= end_datetime:
            filtered_lines_incidents.append(entry['lines'])

    from collections import Counter


    # Flatten the list of lists
    flattened_list = [item for sublist in filtered_lines_incidents for item in sublist]
    # Calculate the frequency of each element
    element_frequencies = Counter(flattened_list)
    # Get the top 10 most common elements and their frequencies
    top_10 = element_frequencies.most_common(10)

    return top_10

# Define the function to filter the accident time data
def get_accident_time_sum(data, start_time, end_time):
    date_format = "%Y-%m-%d %H:%M:%S"
    start_datetime = datetime.strptime(start_time, date_format)
    end_datetime = datetime.strptime(end_time, date_format)

    filtered_accident_sum = {"Verkehrsunfall": 0, "Falschparker": 0}
    #for entry in data['incidents']:
    for entry in data['incidents']:
        # Data validation for the 'start' and 'end' timestamps
        if 'start' not in entry or 'end' not in entry:
            #print("Missing 'start' or 'end' in entry:", entry)
            continue
        if not entry['start'] or not entry['end']:
            #print("Empty 'start' or 'end' timestamp in entry:", entry)
            continue

        try:
            start_incident_datetime = datetime.strptime(entry['start'], date_format)
            end_incident_datetime = datetime.strptime(entry['end'], date_format)
        except ValueError as e:
            #print(f"Error parsing timestamp in entry {entry}: {e}")
            continue

        # Check the incident title and compute the time sum
        if start_datetime <= start_incident_datetime <= end_datetime and ("Verkehrsunfall" in entry['title'] or "Falschparker" in entry['title']):
            if "Verkehrsunfall" in entry['title']:
                filtered_accident_sum['Verkehrsunfall'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60
            if "Falschparker" in entry['title']:
                filtered_accident_sum['Falschparker'] += (end_incident_datetime - start_incident_datetime).total_seconds() / 60

    filtered_accident_sum = {key: (round(value, 2) if isinstance(value, float) else value) for key, value in filtered_accident_sum.items()}

    return filtered_accident_sum


# Define the function to generate the date ranges
def get_date_ranges():
    # Initialize the end date to today's date at 00:00:00
    end_time = datetime.strptime('2023-08-16 00:00:00.00000','%Y-%m-%d %H:%M:%S.%f')
    # Define the time interval (1 week = 7 days)
    interval = timedelta(days=1)
    date_ranges = []
    # Generate the ranges
    for _ in range(3):
        start_time = end_time - interval
        date_ranges.append((start_time, end_time))
        end_time = start_time
    return date_ranges


# Define the function to generate the Subway_line data
def get_line_data(date_ranges):
    with open('hypotheise_1.json', 'r') as file:
        data = json.load(file)

    #top 10 line data
    lines_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filetered_top_10_line_data = get_top10_lines(data, start_str, end_str)
        #print("Line data from data analysis:")
        lines_data_list.append(filetered_top_10_line_data)
    print(lines_data_list)
    return plot_line_data_to_html(lines_data_list)


# Define the function to generate the accident data
def get_accident_data(date_ranges):
    with open('hypotheise_1.json', 'r') as file:
        data = json.load(file)

    #accident time sum daata
    accident_data_list = []
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        filtered_accident_time_data = get_accident_time_sum(data, start_str, end_str)
        #print("Filtered Accident time data")
        accident_data_list.append(filtered_accident_time_data)
    print(accident_data_list)
    return plot_accident_data_to_html(accident_data_list)


# Define the function to generate the gpt data
def get_gpt_data(date_ranges):
    # Use the list of tuples in a for loop
    data = read_data()
    for start, end in date_ranges:
        start_str = start.strftime('%Y-%m-%d %H:%M:%S')
        end_str = end.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Start: {start_str}, End: {end_str}")
        filtered_gpt_data = get_incident_data(data, start_str, end_str)
        
        print("Filtered GPT station data:")
        print(filtered_gpt_data)
        break
    return plot_gpt_data_to_html(filtered_gpt_data)

# Define the function to read the data
def read_data():
    with open('hypotheise_1.json', 'r') as file:
        data = json.load(file)
    return data 
#get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())


In [28]:
get_line_data(get_date_ranges())
#get_accident_data(get_date_ranges())
#get_gpt_data(get_date_ranges())

[[('D', 2), ('U1', 2), ('30A', 2), ('U6', 2), ('42', 1), ('26', 1), ('U4', 1), ('57A', 1), ('60', 1), ('36B', 1)], [('U3', 4), ('49', 2), ('U6', 2), ('6', 1), ('11', 1), ('77A', 1), ('10A', 1), ('1A', 1), ('35A', 1), ('60', 1)], [('2', 3), ('12A', 2), ('43', 2), ('U6', 2), ('D', 2), ('71', 2), ('52', 2), ('60', 2), ('49', 2), ('U1', 1)]]
Generating plot...


In [29]:
get_accident_data(get_date_ranges())

[{'Verkehrsunfall': 173.28, 'Falschparker': 132.45}, {'Verkehrsunfall': 311.37, 'Falschparker': 130.22}, {'Verkehrsunfall': 399.5, 'Falschparker': 33.68}]


# 2023-08-16 hypo-1 tem-0.7

In [31]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.5

In [26]:
get_gpt_data(get_date_ranges())  

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.6

In [24]:
get_gpt_data(get_date_ranges()) 

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.4

In [66]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0178e f\u00c3\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u00b6rung dauert voraussichtlich bis 00:15 Uhr! 3. Update (23:44): Nach einer Fahrtbehinderung kommt es auf der Linie D zu unterschiedlichen Intervallen."}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u00b6rung dauert voraussichtlich bis 19:00 Uhr! 3. Update (18:59): Nach einer Fahrtbehinderung kommt es auf der Linie 42 zu unterschiedlichen Intervallen."}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u00a4\u00c3\u0178ig fahre

# 2023-08-16 hypo-1 tem-0.3

In [33]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.2

In [35]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.1

In [37]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.8

In [39]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.9

In [42]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-1.0

In [47]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf

# 2023-08-16 hypo-1 tem-0.0

In [50]:
get_gpt_data(get_date_ranges())

Start: 2023-08-15 00:00:00, End: 2023-08-16 00:00:00
[{"title": "D: Fremder Verkehrsunfall", "description": "1. Wegen eines fremden Verkehrsunfalles im Bereich Zahnradbahnstra\u00c3\u0192\u00c2\u0178e f\u00c3\u0192\u00c2\u00a4hrt die Linie D nur zwischen Absberggasse und Liechtenwerder Platz. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 00:15 Uhr!"}, {"title": "42: Falschparker", "description": "1. Wegen eines Falschparkers im Bereich Kreuzgasse 21 ist die Linie 42 in Fahrtrichtung Schottentor U an der Weiterfahrt gehindert. 2. Die St\u00c3\u0192\u00c2\u00b6rung dauert voraussichtlich bis 19:00 Uhr!"}, {"title": "26: Schadhaftes Fahrzeug", "description": "1. Die Stra\u00c3\u0192\u00c2\u0178enbahnlinie 26 kann wegen eines schadhaften Fahrzeuges derzeit in beiden Richtungen nur unregelm\u00c3\u0192\u00c2\u00a4\u00c3\u0192\u00c2\u0178ig fahren."}, {"title": "U4: Verunreinigung", "description": "1. Wegen einer Verunreinigung in der Station L\u00c3\u0192\u00c2\u00a4ngenf